## 檢查是否有最新一集

In [64]:
import requests
res = requests.get('https://www.comicbus.com/html/103.html')
res.encoding = 'cp950'
#res.text

In [68]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'lxml')
comics = []
for rec in soup.select('#rp_ctl05_0_dl_0 td a'):
    comics.append(int(rec.get('id').strip('c')))

In [70]:
max(comics)

959

In [72]:
import pandas
comic_df = pandas.DataFrame(comics)
comic_df.columns = ['series']

In [73]:
import sqlite3 as lite
with lite.connect('comics.sqlite') as db:
    comic_df.to_sql('comics', con = db, if_exists='replace')

In [75]:
with lite.connect('comics.sqlite') as db:
    df2 = pandas.read_sql_query('select max(series) as max_series from comics', con = db)
df2

,max_series
0,959


In [81]:
def check_comics():
    # 檢查資料庫最新的漫畫集數
    with lite.connect('comics.sqlite') as db:
        df2 = pandas.read_sql_query('select max(series) as max_series from comics', con = db)
    max_series = df2['max_series'][0]

    # 利用網路爬蟲檢查網路上最新的集數
    res = requests.get('https://www.comicbus.com/html/103.html')
    res.encoding = 'cp950'
    soup = BeautifulSoup(res.text, 'lxml')
    comics = []
    for rec in soup.select('#rp_ctl05_0_dl_0 td a'):
        comics.append(int(rec.get('id').strip('c')))
        
    # 將還未抓取的集數放到 left_comics 之中
    left_comics = []
    for i in range(max_series, max(comics)):
        left_comics.append(i)
        
    # 將抓取到的集數，寫到資料庫中
    with lite.connect('comics.sqlite') as db:
        comic_df.to_sql('comics', con = db, if_exists='replace')
    return left_comics

check_comics()

[]

## 完整漫畫抓取函式

In [82]:
import time, os
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

def getComic(s):
    driver = webdriver.Chrome('/opt/local/common/chromedriver')
    if not os.path.exists(str(s)):
        os.mkdir(str(s))
    pageurl = 'https://www.8899.click/online/comic-103.html?ch={}-{}'
    driver.get(pageurl.format(s,1))
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    pagenum = int(soup.select_one('#pagenum').text.split('/')[1].strip('頁')) 
    for i in range(pagenum):
        driver.get(pageurl.format(s, i+1))
        soup = BeautifulSoup(driver.page_source, 'lxml')
        imgurl = 'https:' + soup.select_one('#TheImg').get('src')
        res = requests.get(imgurl)
        with open('{}/{}.jpg'.format(s, i+ 1), 'wb') as f:
            f.write(res.content)
        time.sleep(1)
    driver.close()

In [84]:
left_comics = check_comics()
left_comics = [959]
for i in left_comics:
    getComic(i)

## 發送漫畫

In [85]:
token = 'VXRv1MIZgkZTKZYn6ykHTof6esBlpXjvVZSjmSvso7s'
headers = {
    'Authorization': f'Bearer {token}'
}

payload = {
 'message':'最新的漫畫', 
}

files = {
    'imageFile': open('959/1.jpg', 'rb')
}

res = requests.post('https://notify-api.line.me/api/notify', data = payload, files=files, headers = headers)

In [41]:
res.text

'{"status":200,"message":"ok"}'

In [86]:
def send_comics(f):
    token = 'VXRv1MIZgkZTKZYn6ykHTof6esBlpXjvVZSjmSvso7s'
    headers = {
        'Authorization': f'Bearer {token}'
    }

    payload = {
     'message':'最新的漫畫', 
    }

    files = {
        'imageFile': open(f, 'rb')
    }

    res = requests.post('https://notify-api.line.me/api/notify', data = payload, files=files, headers = headers)
    return res.json()

## 將完整流程串接

In [ ]:
import os
left_comics = check_comics()
#left_comics = [959]
for i in left_comics:
    getComic(i)
    
    ary = []
    for f in os.listdir(str(i)):
        ary.append(int(f.replace('.jpg', '')))
    ary.sort()
    
    for c in ary:
        #print('{}/{}.jpg'.format(i,c))
        send_comics('{}/{}.jpg'.format(i,c))